# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [1]:
import twitter

CONSUMER_KEY = "NIdJmEzKanvYMoZMQOLRIWGhu"
CONSUMER_SECRET = "OJVCVs1sG5RxOR1XRn30rj2x5BoPZvPzmSmA8kfLMBr2JjH5yZ"

ACCESS_TOKEN_KEY = "105892440-hiutXI6zWd1XjrQJaotg7GbW6Mt1gihXCnE4njZH"
ACCESS_TOKEN_SECRET = "RxIHlIylRycp8dPZfV8fXSM2WtMP74lteIp5P6jxwh4XW"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. 

In [2]:
def get_user_tweets(user_id):
    pass

In [3]:
def get_user_tweets(user_id):
    try:
        timeline = api.GetUserTimeline(user_id=user_id, trim_user=True, exclude_replies=True, count=200)
    except twitter.TwitterError as ex:
        if ex.message == u"Not authorized.":
            return None
        
        if ex.message[0]['code'] == 88:
            sleep_time = api.GetSleepTime("statuses/user_timeline")
            print "sleep for ", sleep_time
            time.sleep(sleep_time)
            timeline = api.GetUserTimeline(user_id=user_id, trim_user=True, exclude_replies=True, count=200)
            
    tweets = []
    for item in timeline:
        if item.lang != 'en':
            continue

        if item.retweeted:
            continue

        if item.truncated:
            continue

        if item.user_mentions:
            continue

        if item.media:
            continue

        if item.urls:
            continue
        
        if 'http' in item.text:
            continue
            
        tweets.append(item.AsDict())
        
    if len(tweets) > 10:
        return tweets
    
    return None
    

## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). 

In [4]:
def get_words(text):
    pass

In [5]:
import nltk

pattern = "(?x)([A-Z]\.)+|\$?\d+(\.\d+)?%?|\w+([-']\w+)*|[+/\-@&*]"


def get_words(text):
    words = [t.lower() for t in nltk.regexp_tokenize(text, pattern)]
    return words

Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

In [6]:
import nltk
nltk.download()

KeyboardInterrupt: 

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

In [7]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
wnl = nltk.WordNetLemmatizer()

word = 'WORDS'
word_nf = wnl.lemmatize(word.lower())
print word_nf
if word_nf in stop:
    print "This is stopword"
else:
    print "This is not stopword"

word
This is not stopword


Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [8]:
def get_tokens(words):
    pass

In [9]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
wnl = nltk.WordNetLemmatizer()

def get_tokens(words):
    tokens = [wnl.lemmatize(t.lower()) for t in words]
    return [i for i in tokens if i not in stop]

Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита.

In [10]:
def get_tweet_tokens(tweet):
    pass

In [11]:
import nltk
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
wnl = nltk.WordNetLemmatizer()

pattern = "(?x)([A-Z]\.)+|\$?\d+(\.\d+)?%?|\w+([-']\w+)*|[+/\-@&*]"
email_re = re.compile(r'[\w\-][\w\-\.]+@[\w\-][\w\-\.]+[a-zA-Z]{1,4}')
nick_re = re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-\.]))@([A-Za-z_]+[A-Za-z0-9_]+)')

def get_tweet_tokens(tweet):
    text = tweet['text']
    for email in re.findall(email_re, text):
        if email.startswith('//'):
            continue
        text = text.replace(email, 'email_token')
        
    for nick in re.findall(nick_re, text):
        text = text.replace('@'+nick, 'nick_token')
    words = get_words(text)
    return get_tokens(words)
 


Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [12]:
def collect_users_tokens():
    pass

In [16]:
import pandas as pd
from collections import defaultdict
import json
from sklearn.feature_extraction import DictVectorizer

def collect_users_tokens(): 
    TRAINING_SET_URL = "twitter_train.csv"
    df_users = pd.read_csv(TRAINING_SET_URL, sep="\t", header=None, names=["twitter_id", "is_1", "is_2", "is_3"])
    
    user_tweets = open("user_tweets.txt", "w")
    
    vectorizer = DictVectorizer()
    
    users = []
    
    for index, row in df_users.iterrows():
        user_id = row['twitter_id']
        tweets = get_user_tweets(user_id)
        if not tweets:
            continue
            
        user_tweets.write(json.dumps((user_id, tweets)))
        user_tweets.write('\n')
        
        
        user_to_tokens = defaultdict(int)
        
        for tweet in tweets:
            tokens = get_tweet_tokens(tweet)
            print tokens
            if not tokens:
                continue
            for token in tokens:
                user_to_tokens[token] += 1
        
        users.append(user_to_tokens)
        
        if index > 1:
            break
        
    user_tweets.close()
    
    vectorized_sparse = vectorizer.fit_transform(users)
    print vectorized_sparse
    
collect_users_tokens()


[u'american', u"mcgee's", u'alice', u'ost', u'perfect', u'usually']
[u'alice', u'madness', u'return', u'completed', u'twas', u'brillig', u'alice']
[u'brittany', u'baby', u'rain', u'weather']
[u'wireless', u'headphone', u'much', u'fun', u'-', u'-', u'peed', u'listening', u'daft', u'punk']
[u'crossing', u'finger', u'jetbrains']
[u'love', u'power', u'metal', u'happily', u'epic', u'inspired', u'dragonforce']
[u'listening', u'equilibrium', u'using', u'pink', u'headphone', u"i'm", u'true']
[u'wire', u'anything', u'directly', u'anything', u"i'm", u'professor', u'c', u'futurama']
[u'alert', u'debugging']
[u'worked', u'congratulation', u'first', u'django-powered', u'page']
[u'remember', u'back', u"wasn't", u'wikipedia', u'dark', u'age', u'man', u'c', u'eric', u'lippert']
[u'american', u"mcgee's", u'alice', u'ost', u'perfect', u'usually']
[u'alice', u'madness', u'return', u'completed', u'twas', u'brillig', u'alice']
[u'brittany', u'baby', u'rain', u'weather']
[u'wireless', u'headphone', u'much',

KeyboardInterrupt: 